### GERAADA Score Automation Script
This script takes 'input.csv' from the folder this script is located in, runs each row through the Shiny app located at https://web.imbi.uni-heidelberg.de/geraada-score/, and then returns the output (patient ID, input variables, and risk score) to 'output.csv' located in the folder this script is in.

##### Notes on WebDriver
ChromeDriver needs to be aligned with Chrome version (paste chrome://version/ in your Chrome browser to locate)
<br> Once version is located, download the ChromeDriver that matches that version from here https://chromedriver.chromium.org/
<br> Once downloaded, place on your computer in a dedicated spot and copy path into the code below
<br> This allows the python script to open Chrome and run through the app automatically.
<br> There are also drivers available for Firefox, Edge, and Safari if needed, but the code would need adjusted.

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import csv
import os

class element_has_non_empty_text(object):
    """An expectation for checking that an element has non-empty text.

    locator - used to find the element
    returns the WebElement once it has the non-empty text
    """
    def __init__(self, locator):
        self.locator = locator

    def __call__(self, driver):
        element = driver.find_element(*self.locator)   # Finding the referenced element
        if element.text.strip() != "":
            return element
        else:
            return False

# Function to handle Selectize dropdowns by clearing the default value and inputting the new value
def handle_selectize_dropdown(dropdown_id, value):
    input_id = dropdown_id + "-selectized"
    dropdown_input = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, input_id)))
    
    # Clear existing/default value
    dropdown_input.click()
    dropdown_input.send_keys(Keys.CONTROL + 'a')
    dropdown_input.send_keys(Keys.BACKSPACE)
    
    # Enter the new value
    dropdown_input.send_keys(value)
    dropdown_input.send_keys(Keys.ENTER)  # Press ENTER to confirm the selection

# Setup Selenium with Chrome
chrome_driver_path = "C:\\Users\\user\\OneDrive\\Desktop\\ChromeDriver\\chromedriver.exe" # Adjust path as needed
service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service)

# Define the output CSV file path and check if it needs headers
output_csv_path = 'output.csv'
file_exists = os.path.isfile(output_csv_path) and os.path.getsize(output_csv_path) > 0

# Read input data from CSV
input_df = pd.read_csv('input.csv', dtype={'PATID': str})

for index, row in input_df.iterrows():
    try:
        driver.get('https://web.imbi.uni-heidelberg.de/geraada-score/')
        wait = WebDriverWait(driver, 30)

        # Fill out the form
        age_element = wait.until(EC.presence_of_element_located((By.ID, 'opAge')))
        age_element.click()
        age_element.send_keys(Keys.CONTROL + 'a')
        age_element.send_keys(str(row['AGE']))

        # Example usage of handling selectize dropdowns
        handle_selectize_dropdown('sex', row['SEX'])
        handle_selectize_dropdown('reanimation', row['RESUCITATION_BEFORE_SURGERY'])
        handle_selectize_dropdown('HerzOP', row['PREVIOUS_CARDIAC_SURGERY'])
        handle_selectize_dropdown('intubiert', row['INTUBATION_VENTILATION_AT_REFERRAL'])
        handle_selectize_dropdown('katecholamine', row['CATECHOLAMINES_AT_REFERRAL'])
        handle_selectize_dropdown('aortenklappeninsuffizienz', row['AORTIC_VALVE_REGURGITATION'])
        handle_selectize_dropdown('hemiparesePrae', row['PREOPERATIVE_HEMIPARESIS'])
        handle_selectize_dropdown('entryBogen', row['LOCATION_OF_PRIMARY_ENTRY_TEAR_WITHIN_AORTIC_ARCH'])

        # Handle checkboxes
        # Handle malperfusion checkboxes
        default_malperfusion_checkbox = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"input[name='malperfusionPrae'][value='no']")))
        if default_malperfusion_checkbox.is_selected():
            driver.execute_script("arguments[0].checked = false;", default_malperfusion_checkbox)

        malperfusion_values = row['PREOPERATIVE_ORGAN_MALPERFUSION'].split(',')
        for value in malperfusion_values:
            value = value.strip()  # remove any leading or trailing spaces
            malperfusion_checkbox = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"input[name='malperfusionPrae'][value='{value}']")))
            if not malperfusion_checkbox.is_selected():
                malperfusion_checkbox.click()

        # Handle dissection checkboxes
        default_dissection_checkbox = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"input[name='dissection'][value='unknown or other']")))
        if default_dissection_checkbox.is_selected():
            driver.execute_script("arguments[0].checked = false;", default_dissection_checkbox)

        dissection_values = row['EXTENSION_OF_DISSECTION'].split(',')
        for value in dissection_values:
            value = value.strip()  # remove any leading or trailing spaces
            dissection_checkbox = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"input[name='dissection'][value='{value}']")))
            if not dissection_checkbox.is_selected():
                dissection_checkbox.click()

        # Simulated wait for processing (replace with actual interaction wait times)
        time.sleep(5)
        
        # Click submit button
        wait.until(EC.presence_of_element_located((By.ID, 'button'))).click()

        risk_element = WebDriverWait(driver, 120).until(element_has_non_empty_text((By.ID, 'RiskScore')))
        risk = risk_element.text.split()[-1] if risk_element.text.split() else "Unknown"

        # Write result to CSV file
        with open(output_csv_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            if not file_exists:
                writer.writerow(['PATID', 'AGE', 'SEX', 'RESUCITATION_BEFORE_SURGERY', 'PREVIOUS_CARDIAC_SURGERY',
                                 'INTUBATION_VENTILATION_AT_REFERRAL', 'CATECHOLAMINES_AT_REFERRAL', 
                                 'AORTIC_VALVE_REGURGITATION', 'PREOPERATIVE_ORGAN_MALPERFUSION', 
                                 'PREOPERATIVE_HEMIPARESIS', 'EXTENSION_OF_DISSECTION', 'LOCATION_OF_PRIMARY_ENTRY_TEAR_WITHIN_AORTIC_ARCH',
                                 'Risk']) 
                file_exists = True
            writer.writerow([row['PATID'], row['AGE'], row['SEX'],
                             row['RESUCITATION_BEFORE_SURGERY'], row['PREVIOUS_CARDIAC_SURGERY'],
                                row['INTUBATION_VENTILATION_AT_REFERRAL'], row['CATECHOLAMINES_AT_REFERRAL'], 
                                 row['AORTIC_VALVE_REGURGITATION'], row['PREOPERATIVE_ORGAN_MALPERFUSION'], 
                                 row['PREOPERATIVE_HEMIPARESIS'], row['EXTENSION_OF_DISSECTION'], row['LOCATION_OF_PRIMARY_ENTRY_TEAR_WITHIN_AORTIC_ARCH'],
                                 risk])

    except Exception as e:
        print(f"Failed on PATID: {row['PATID']}. Error: {e}")

        # Logging failed PATID
        with open('failed_patids.csv', mode='a', newline='') as fail_file:
            fail_writer = csv.writer(fail_file)
            fail_writer.writerow([row['PATID']])
        # Optionally, you can break or continue based on your requirement

driver.quit()
